In [1]:
import yaml

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from dataset import CIFAR10
from trainer import Trainer
from transform import get_transform
from model import ResNet
import scheduler

In [2]:
with open('configs/train.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
train_transform, test_transform = get_transform()

trainset = CIFAR10(train=True, transform=train_transform, **config['dataset'])
testset = CIFAR10(train=False, transform=test_transform, **config['dataset'])

trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, drop_last=True, **config['dataloader'])
testloader = torch.utils.data.DataLoader(testset, shuffle=False, drop_last=False, **config['dataloader'])

In [4]:
trainer = Trainer(**config['trainer'])

In [5]:
def get_layer_config(n):
    return [(16, 1, n), (32, 2, n), (64, 2, n)]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

for model_name, model_config in config['experiments'].items():    
    model = ResNet(layer_config=get_layer_config(model_config['n']), **config[model_config['model']])
    first_in_channels = config[model_config['model']]['first_in_channels']
    model.conv1 = nn.Sequential(nn.Conv2d(3, first_in_channels, kernel_size=3, stride=1, padding=1, bias=False),
                                        nn.BatchNorm2d(first_in_channels),
                                        nn.ReLU(inplace=True))
    model.to(device)
    
    if 'params' in config['criterion']:
        criterion = getattr(nn, config['criterion']['name'])(**config['criterion']['params'])
    else:
        criterion = getattr(nn, config['criterion']['name'])()

    optimizer = getattr(optim, config['optimizer']['name'])(model.parameters(), **config['optimizer']['params'])

    lr_scheduler = getattr(scheduler , config['scheduler']['name'])(optimizer, **config['scheduler']['params'])
    
    trainer.train(model_name, model, config['n_epochs'], trainloader, testloader, criterion, optimizer, lr_scheduler)

100%|██████████| 164/164 [18:57<00:00,  6.93s/it]


Finished training plain-20
  train_error : 1.2%
  test_error  : 9.4%



100%|██████████| 164/164 [29:22<00:00, 10.75s/it]


Finished training plain-32
  train_error : 1.3%
  test_error  : 9.3%



100%|██████████| 164/164 [39:41<00:00, 14.52s/it]


...

Finished training resnet-44
  train_error : 0.1%
  test_error  : 6.7%



100%|██████████| 164/164 [51:33<00:00, 18.86s/it]


Finished training resnet-56
  train_error : 0.1%
  test_error  : 6.8%



 25%|██▌       | 41/164 [4:56:08<14:48:25, 433.38s/it]


KeyboardInterrupt: 

In [ ]:
optimizer.param_groups[0]['lr']

In [ ]:
import numpy as np

In [ ]:
n_epochs = 20
name = 'resnet20'

In [ ]:
trainer = Trainer(42, 'logs', 'weights')

In [ ]:
trainer.train(name, model, n_epochs, trainloader, testloader, criterion, optimizer, lr_scheduler)

In [ ]:
#plt.plot(train_loss)
plt.plot(1 - np.array(train_acc))
#plt.plot(val_loss)
plt.plot(1 - np.array(val_acc))
plt.ylim(0, 0.2)
plt.show()

In [ ]:
pred = torch.LongTensor([1,2,3]).cuda()
labels = torch.LongTensor([1,1,1]).cuda()
c = torch.sum(pred == labels)
c

In [ ]:
c.shape

In [ ]:
len(trainloader)

In [ ]:
len(trainloader.dataset)

In [ ]:
best_score = float('inf')

for epoch in range(1, n_epochs+1):
    
    # Training Phase

    running_loss = 0.0
    running_correct = 0
    running_data = 0

    model.train()
    
    for inputs, labels in tqdm(trainloader):
        
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        with torch.no_grad():
            pred = torch.argmax(outputs, dim=-1)
            running_correct += torch.sum(pred == labels)
            running_loss += loss * inputs.size(0)
            running_data += inputs.size(0)

    train_epoch_loss = running_loss / running_data
    train_acc = running_correct / running_data

    
    # Validation Phase
    
    running_loss = 0.0
    running_correct = 0
    running_data = 0

    model.eval()
    
    for inputs, labels in tqdm(testloader):
        
        inputs, labels = inputs.cuda(), labels.cuda()
        
        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            pred = torch.argmax(outputs, dim=-1)
            running_correct += torch.sum(pred == labels)
            running_loss += loss * inputs.size(0)
            running_data += inputs.size(0)
    
    test_epoch_loss = running_loss / running_data
    test_epoch_acc = running_correct / running_data
    
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    val_loss.append(test_epoch_loss)
    val_acc.append(test_epoch_acc)
    
    if test_epoch_loss < best_score:
        torch.save(model.state_dict(), 'save/n3.pt')
        best_score = test_epoch_loss
#     else:
#         if patience_cnt == patience:
#             break
#         patience_cnt += 1

print('Finished Training')